In [0]:
from pyspark.sql.functions import *


  # Azure Event Hub Configuration
event_hub_namespace='hospital-analytics-mn'
event_hub_name='hospital-analytis-eh'
event_hub_conn_str=dbutils.secrets.get(scope='hospitalAnalyticsVaultScope', key='eventhub-connection-string')

kafka_options = {
    'kafka.bootstrap.servers':f'{event_hub_namespace}.servicebus.windows.net:9093',
      'subscribe': event_hub_name,
          'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{event_hub_conn_str}";',
          'kafka.sasl.mechanism': 'PLAIN',
          'kafka.security.protocol':  'SASL_SSL',
          'startingOffsets': 'latest',
          'failOnDataLoss': 'false'
}

# Read form event hub
raw_df = (spark.readStream.format('kafka')
  .options(**kafka_options)
  .load()
)

# Cast data to json
json_df = raw_df.selectExpr('CAST(value AS STRING) as raw_json')

# ADLS configuration
# We can use either key or string based access to accees the azure storage account (Using key based access)
spark.conf.set( 
               "fs.azure.account.key.hospitalstorage224.dfs.core.windows.net",
  dbutils.secrets.get(scope='hospitalAnalyticsVaultScope', key='storage-connection'))

bronze_path = "abfss://bronze@hospitalstorage224.dfs.core.windows.net/patient_flow"

# Write stream to bronze
(
    json_df
    .writeStream
    .format("delta")
    .option("checkpointLocation", "dbfs:/mnt/bronze/_checkpoints/patient_flow")
    .start(bronze_path)
)

In [0]:
df_bronze=spark.read.format('delta').load(bronze_path)
display(df_bronze)

raw_json
"{""patient_id"": ""747eedce-d0cf-4809-b67d-9ee2d3aeab5b"", ""gender"": ""Other"", ""age"": 60, ""department"": ""ICU"", ""admission_time"": ""2025-12-13T16:32:48.167302"", ""discharge_time"": ""2025-12-14T21:32:48.167302"", ""bed_id"": 167, ""hospital_id"": 7, ""event_timestamp"": ""2025-12-13T20:32:48.169861"", ""event_type"": ""discharge""}"
"{""patient_id"": ""b5cb49a2-e2b3-47cc-b67e-cc0cab1278b1"", ""gender"": ""Other"", ""age"": 68, ""department"": ""Cardiology"", ""admission_time"": ""2025-12-13T03:32:50.202126"", ""discharge_time"": ""2025-12-16T03:32:50.202126"", ""bed_id"": 365, ""hospital_id"": 1, ""event_timestamp"": ""2025-12-13T20:32:50.202126"", ""event_type"": ""discharge""}"
"{""patient_id"": ""c16aa8b1-c6b1-45d1-82c5-6bee5b3abc96"", ""gender"": ""Female"", ""age"": 31, ""department"": ""Pediatrics"", ""admission_time"": ""2025-12-15T03:32:50.941256"", ""discharge_time"": ""2025-12-11T23:32:50.941256"", ""bed_id"": 403, ""hospital_id"": 1, ""event_timestamp"": ""2025-12-13T20:32:50.941256"", ""event_type"": ""discharge""}"
"{""patient_id"": ""ca144647-8592-40f1-873d-72781516d772"", ""gender"": ""Other"", ""age"": 66, ""department"": ""Cardiology"", ""admission_time"": ""2025-12-11T00:32:53.240095"", ""discharge_time"": ""2025-12-12T04:32:53.240095"", ""bed_id"": 104, ""hospital_id"": 7, ""event_timestamp"": ""2025-12-13T20:32:53.240095"", ""event_type"": ""admission""}"
"{""patient_id"": ""332375b4-5724-46af-8043-b9cdd5ffa9f3"", ""gender"": ""Female"", ""age"": 47, ""department"": ""ICU"", ""admission_time"": ""2025-12-11T10:32:54.113006"", ""discharge_time"": ""2025-12-12T10:32:54.113006"", ""bed_id"": 354, ""hospital_id"": 2, ""event_timestamp"": ""2025-12-13T20:32:54.113006"", ""event_type"": ""transfer""}"
"{""patient_id"": ""6dc35d63-2045-4b2c-925b-058cac6bfb23"", ""gender"": ""Male"", ""age"": 72, ""department"": ""Maternity"", ""admission_time"": ""2025-12-13T19:32:55.104849"", ""discharge_time"": ""2025-12-16T13:32:55.104849"", ""bed_id"": 357, ""hospital_id"": 4, ""event_timestamp"": ""2025-12-13T20:32:55.104849"", ""event_type"": ""admission""}"
"{""patient_id"": ""f834fac0-3666-46f5-ba07-c799d621c3fd"", ""gender"": ""Other"", ""age"": 22, ""admission_time"": ""2025-12-14T21:32:57.466644"", ""discharge_time"": ""2025-12-14T22:32:57.466644"", ""bed_id"": 289, ""hospital_id"": 6, ""event_timestamp"": ""2025-12-13T20:32:57.466644"", ""event_type"": ""transfer""}"
"{""patient_id"": ""f7596a23-58e1-4483-a250-70b889ec9d92"", ""gender"": ""Other"", ""age"": 100, ""department"": ""Pediatrics"", ""admission_time"": ""2025-12-12T01:32:59.275648"", ""discharge_time"": ""2025-12-12T02:32:59.275648"", ""bed_id"": 255, ""hospital_id"": 3, ""event_timestamp"": ""2025-12-13T20:32:59.275648"", ""event_type"": ""discharge""}"
"{""patient_id"": ""e81b54d4-6b2a-4240-a096-ac1475a4147a"", ""gender"": ""Female"", ""age"": 6, ""department"": ""Cardiology"", ""admission_time"": ""2025-12-12T22:33:00.171301"", ""discharge_time"": ""2025-12-15T13:33:00.171301"", ""bed_id"": 416, ""hospital_id"": 1, ""event_timestamp"": ""2025-12-13T20:33:00.171301"", ""event_type"": ""discharge""}"
"{""patient_id"": ""99b58922-96df-49c1-a4c7-d00b2ec07d73"", ""gender"": ""Other"", ""age"": 25, ""department"": ""Emergency"", ""admission_time"": ""2025-12-12T19:33:01.743391"", ""discharge_time"": ""2025-12-13T00:33:01.743391"", ""bed_id"": 249, ""hospital_id"": 1, ""event_timestamp"": ""2025-12-13T20:33:01.743391"", ""event_type"": ""transfer""}"
